<a href="https://colab.research.google.com/github/Vicks-2019/Pytorch/blob/main/Image_classification_using_CNN_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from torchvision import transforms
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

In [3]:
_tasks = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

In [4]:
## load the dataset 
from torchvision.datasets import CIFAR10
cifar = CIFAR10('data', train=True, download=True, transform=_tasks)

Extracting data/cifar-10-python.tar.gz to data


In [5]:
## create training and validation split 
split = int(0.8 * len(cifar))
index_list = list(range(len(cifar)))
train_idx, valid_idx = index_list[:split], index_list[split:]

In [7]:
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [8]:
## create training and validation sampler objects
tr_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(valid_idx)

In [9]:
## create iterator objects for train and valid datasets
trainloader = DataLoader(cifar, batch_size=256, sampler=tr_sampler)
validloader = DataLoader(cifar, batch_size=256, sampler=val_sampler)

In [12]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        ## define the layers
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.linear1 = nn.Linear(1024, 512)
        self.linear2 = nn.Linear(512, 10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 1024) ## reshaping 
        x = F.relu(self.linear1(x))
        x = self.linear2(x)
        return x

model = Model()

In [13]:
import torch.optim as optim
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay= 1e-6, momentum = 0.9, nesterov = True)

In [14]:
## run for 30 Epochs
for epoch in range(1, 31):
    train_loss, valid_loss = [], []

In [15]:
 ## training part 
model.train()
for data, target in trainloader:
  optimizer.zero_grad()
  output = model(data)
  loss = loss_function(output, target)
  loss.backward()
  optimizer.step()
  train_loss.append(loss.item()) 

In [16]:
 ## evaluation part 
model.eval()
for data, target in validloader:
  output = model(data)
  loss = loss_function(output, target)
  valid_loss.append(loss.item())

In [17]:
## dataloader for validation dataset 
dataiter = iter(validloader)
data, labels = dataiter.next()
output = model(data)

In [19]:
import numpy as np
_, preds_tensor = torch.max(output, 1)
preds = np.squeeze(preds_tensor.numpy())

In [20]:
print ("Actual:", labels[:10])
print ("Predicted:", preds[:10])

Actual: tensor([3, 2, 4, 2, 4, 6, 2, 6, 8, 9])
Predicted: [0 4 2 5 4 7 4 6 8 0]
